<a href="https://colab.research.google.com/github/VictorMadu/Fraud-Detection-Challenge/blob/master/Machine%20learning%20and%20stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import os
from scipy import stats
import pickle
import warnings; warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv("/content/drive/My Drive/fraud detection/data/processed_train.csv")

In [0]:
print(df.shape[0], df.shape[1])

10493 1259


In [0]:
df.head()

,account destination verification,account source verification,account type,age,credit card type,current bank amount,fradulent,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)
0,1,1,saving,46,master,43671,0,acus0,1151263,married,348764,musician,2299,0,1,2016,4,10,40,5,1,44,58,2,31,4,-1107592,-802499,305093,49890.026256,99917,30,99887,110207068,-0.001333,-1.197369,29083.038329,2209,751193.550023,1499877,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,18.995650,500.766855,151.702479,-481.771205,-349.064376,305093
1,0,1,current,64,master,7442,0,acus1,1062843,single,225114,accounting,4553,0,1,2015,1,1,3,14,15,5,7,2,31,7,-1055401,-837729,217672,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,1.634527,233.437953,49.443005,-231.803426,-183.994948,217672
2,1,1,current,41,master,40102,0,acus2,48042,married,420486,farmer,1493,1,1,2016,4,10,43,28,13

In [0]:
df[df.isnull().any(1)]

,account destination verification,account source verification,account type,age,credit card type,current bank amount,fradulent,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)


In [0]:
cat = ['age', 'account type','account source verification', 'transaction source method', 'account destination verification', 'credit card type',  'age_group', 'marital_status', 'occupation', 'is_busday', 'year', 'quarter', 'month', 'week', 'day', 'hour', 'weekday', 'no_days_in_month', 'minute', 'second']
for i in cat:
  df[i] = df[i].astype('category')

In [0]:
gat =  ['age', 'account type', 'account source verification', 'transaction source method', 'account destination verification', 'credit card type', 'age_group', 'marital_status', 'occupation', 'is_busday', 'year', 'quarter', 'month', 'week', 'day', 'hour', 'weekday', 'no_days_in_month', 'minute', 'second', 'fradulent']
gf = df[gat]
cols_dict = dict()                                 
for col in cat:
  groups = gf.groupby(col)['fradulent'].mean()
  uniques = gf[col].unique()
  uniques_dict = dict(zip(uniques, groups))
  cols_dict[col] = uniques_dict

In [0]:
gf.head()

,age,account type,account source verification,transaction source method,account destination verification,credit card type,age_group,marital_status,occupation,is_busday,year,quarter,month,week,day,hour,weekday,no_days_in_month,minute,second,fradulent
0,46,saving,1,0,1,master,4,married,musician,1,2016,4,10,40,5,1,2,31,44,58,0
1,64,current,1,0,0,master,7,single,accounting,1,2015,1,1,3,14,15,2,31,5,7,0
2,41,current,1,1,1,master,3,married,farmer,1,2016,4,10,43,28,13,4,31,44,58,0
3,66,current,1,1,1,master,8,unknown,clergy,0,2016,4,11,46,20,15,6,30,5,32,0
4,78,current,0,0,1,verve,10,married,accounting,1,2015,3,9,36,1,16,1,30,11,57,1


In [0]:
from sklearn.base import BaseEstimator, TransformerMixin, clone
class MeanEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols=[]):
        self.cols = cols 

    def transform(self, X, **transformparams):
        assert isinstance(X, pd.DataFrame) 
        for col in self.cols:
            X[col+" mean_encoded"] = X[col].map(self.mean_mapped_dict[col])
        return X

        
    def fit(self, X, y=None, **fitparams):
      assert isinstance(X, pd.DataFrame)
      global cols_dict 
      self.mean_mapped_dict = cols_dict.copy()
      return self

In [0]:
from sklearn.utils import resample

def upsample(X, y):
  X = np.array(X).reshape(X.shape[0], X.shape[1])
  y = np.array(y, dtype='int').reshape(y.shape[0], 1)
  assert np.sum(y[y==0]) != np.sum(y[y==1])
  df = np.concatenate((X, y), axis=1)
  if np.unique(y, return_counts=True)[1][0] > np.unique(y, return_counts=True)[1][1]:
    df_majority = df[df[:,-1]==0]
    df_minority = df[df[:,-1]==1]
  else:
    df_majority = df[df[:,-1]==1]
    df_minority = df[df[:,-1]==0]
  
    
  df_minority_upsampled = resample(df_minority, 
                                  replace=True,     # sample with replacement
                                  n_samples=len(df_majority),    # to match majority class
                                  random_state=32) # reproducible results
  
  df_upsampled = np.concatenate((df_majority, df_minority_upsampled))
  
  return df_upsampled[:, :-1], df_upsampled[:,-1]

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight

X = df.drop(['fradulent', 'id'], axis=1)
y = df['fradulent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

w_array_train = compute_sample_weight('balanced', y_train)
w_array = compute_sample_weight('balanced', y)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.feature_selection import SelectKBest

cat_feat = df.drop(['fradulent','id'], axis=1).select_dtypes(exclude=['int', 'float']).columns

num_transform1 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('fs', SelectKBest(k=168)),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    
])

num_transform2 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('fs', SelectKBest(k=168))
])


num_transform3 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('fs', SelectKBest(k=30))
])

num_transform4 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('fs', SelectKBest(k=40))
])

cat_transform1 = Pipeline(steps=[
    ('mean_encoder', MeanEncoder(cols=cat_feat)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    
])

cat_transform2 = Pipeline(steps=[
    ('mean_encoder', MeanEncoder(cols=cat_feat)),
    ('ordinal', OrdinalEncoder())
    
])



In [0]:
from sklearn.compose import ColumnTransformer
from scipy.stats import uniform, truncnorm, randint
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import SelectKBest



num_feat = X.select_dtypes(include=['int', 'float']).columns
cat_feat = X.select_dtypes(exclude=['int', 'float']).columns

preprocesser1 = ColumnTransformer(transformers=[
      ('num', num_transform1, num_feat),
      ('cat', cat_transform1, cat_feat)
  ])

preprocesser2 = ColumnTransformer(transformers=[
      ('num', num_transform2, num_feat),
      ('cat', cat_transform1, cat_feat)
  ])
preprocesser3 = ColumnTransformer(transformers=[
      ('num', num_transform2, num_feat),
      ('cat', cat_transform2, cat_feat)
  ])
preprocesser4 = ColumnTransformer(transformers=[
      ('num', num_transform3, num_feat),
      ('cat', cat_transform1, cat_feat)
  ])

preprocesser5 = ColumnTransformer(transformers=[
      ('num', num_transform4, num_feat),
      ('cat', cat_transform1, cat_feat)
  ])

_ = preprocesser1.fit(X_train, y_train)
_ = preprocesser2.fit(X_train, y_train)
_ = preprocesser3.fit(X_train, y_train)
_ = preprocesser4.fit(X_train, y_train)
_ = preprocesser5.fit(X_train, y_train)

In [0]:
! pip install catboost

     |████████████████████████████████| 64.4MB 47kB/s 


In [0]:
from keras.initializers import RandomNormal, glorot_normal, he_normal, glorot_uniform, he_uniform 
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint


def nn_model1(shape, optimizer='adam', init='glorot_uniform'):
  model = Sequential()
  model.add(Dropout(0.6))
  model.add(Dense(8, kernel_initializer=init, input_shape=(shape,), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(8, kernel_initializer=init, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(8, kernel_initializer=init, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(8, kernel_initializer=init, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model


def nn_model2(shape, optimizer='adam', init='glorot_uniform'):
  model = Sequential()
  model.add(Dropout(0.6))
  model.add(Dense(24, kernel_initializer=init, input_shape=(shape,), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(24, kernel_initializer=init, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  
  model.add(Dense(24, kernel_initializer=init, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model


def nn_model3(shape, optimizer='adam', init='glorot_uniform'):
  model = Sequential()
  model.add(Dropout(0.6))
  model.add(Dense(32, kernel_initializer=init, input_shape=(shape,), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(32, kernel_initializer=init, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))

  model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model


Using TensorFlow backend.


In [0]:
params = dict()
params['learning_rate'] = 0.01
params['num_iterations'] = 600
params['num_leaves'] = 8
params['max_depth'] = -4
params['max_bin'] = 26
params['reg_lambda'] = 13
params['reg_alpha'] = 13
params['min_gain_to_split'] = 4
params['extra_trees'] = True


In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

class_weights = compute_class_weight('balanced', np.unique(y), y)



clf1 = KNeighborsClassifier(weights='distance', n_neighbors=10)
clf2 = KNeighborsClassifier(weights='uniform', n_neighbors=10)
clf3 = LogisticRegression(class_weight='balanced')
clf4 = ExtraTreesClassifier(class_weight='balanced')
clf5 = RandomForestClassifier(class_weight='balanced')
clf6 = BernoulliNB()

clf7 = LGBMClassifier(objective='binary', metric='auc', bagging_freq=1, **params)



clf8 = XGBClassifier(max_depth=3, learning_rate=0.2, n_estimators=1000,  min_child_weight=1,
                    max_delta_step=1, subsample=0.5, colsample_bytree=0.5, colsample_bylevel=0.5, reg_lambda=0.3, reg_alpha=0.0, maxbin=500, eval_metric=['auc'])


clf9 = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.5053062630331221, loss='deviance',
                           max_depth=3, max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=2, min_samples_split=3,
                           min_weight_fraction_leaf=0.0, n_estimators=102,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=0.8929118586409764,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)



clf10 = AdaBoostClassifier()
clf11 = SVC(kernel='linear', probability=True)
clf12 = SVC(kernel='rbf', probability=True)
clf13 = SVC(kernel='poly', probability=True)
clf14 = AdaBoostClassifier(base_estimator=SVC(kernel='linear', probability=True), n_estimators=10)
clf15 = AdaBoostClassifier(base_estimator=SVC(kernel='rbf', probability=True), n_estimators=10)
clf16 = AdaBoostClassifier(base_estimator=SVC(kernel='poly', probability=True), n_estimators=10)
clf17 = AdaBoostClassifier(base_estimator=LogisticRegression(class_weight='balanced'), n_estimators=30)

clf18 = CatBoostClassifier(learning_rate=0.2, iterations=600, eval_metric='AUC', depth=4, l2_leaf_reg=2.0, task_type='GPU', 
                          class_weights=class_weights, bagging_temperature=0.5)

clf19 = nn_model1(8394)
clf20 = nn_model2(8394)
clf21 = nn_model3(8394)


In [0]:
sclf1 = LGBMClassifier(objective='binary', metric=['auc', 'logloss'], bagging_freq=1, **params)

sclf2 = CatBoostClassifier(learning_rate=0.15, iterations=1000, eval_metric='AUC', depth=4, l2_leaf_reg=2.0, task_type='GPU',
                          class_weights=class_weights, bagging_temperature=0.5)


sclf3 = XGBClassifier(max_depth=3, learning_rate=0.15, n_estimators=1000,  min_child_weight=1,
                    max_delta_step=1, subsample=0.5, colsample_bytree=0.5, colsample_bylevel=0.5, reg_lambda=0.3, reg_alpha=0.0, maxbin=500, eval_metric=['auc', 'logloss'])

sclf4 = nn_model1(856)
sclf5 = nn_model2(856)
sclf6 = nn_model3(856)

sclf7 = LogisticRegression(class_weight='balanced')

In [0]:
ssclf1 = LGBMClassifier(objective='binary', metric=['auc', 'Logloss'], bagging_freq=1, **params)

ssclf2 = CatBoostClassifier(learning_rate=0.1, iterations=900, eval_metric='AUC', depth=4, l2_leaf_reg=2.0, task_type='GPU',
                          class_weights=class_weights, bagging_temperature=0.5)


ssclf3 = XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=2000, min_child_weight=1,
                    max_delta_step=1, subsample=0.5, colsample_bytree=0.5, colsample_bylevel=0.5, reg_lambda=0.3, reg_alpha=0.0, maxbin=500, eval_metric=['auc', 'logloss'])

ssclf4 = LogisticRegression(class_weight='balanced')

In [0]:
sclf = CatBoostClassifier(learning_rate=0.7, iterations=3000, eval_metric='AUC', depth=4, l2_leaf_reg=2.0,  task_type='GPU',
                          class_weights=class_weights, bagging_temperature=0.5)

In [0]:
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone
from keras.callbacks import EarlyStopping
from catboost import Pool
from sklearn.model_selection import KFold



def stack_train(x, y, n_splits=5):
  kfold = KFold(n_splits=n_splits, shuffle=True, random_state=156)
  x1 = preprocesser1.transform(x)
  x2 = preprocesser2.transform(x).toarray()
  x3 = preprocesser3.transform(x)
  x4 = preprocesser4.transform(x).toarray()
  x5 = preprocesser5.transform(x).toarray()
  first_operation = True
  class_weights = compute_class_weight('balanced', np.unique(y), y)

print('First stack')
for i, (train_index, val_index) in enumerate(kfold.split(x, y)):
  
  print(f'\n\t{i+1} cv')
  w_array = compute_sample_weight('balanced', y.iloc[train_index])
  x_upsampled, y_upsampled = upsample(x3[train_index,:], y.iloc[train_index])

  clf1.fit(x_upsampled, y_upsampled)
  y_pred = clf1.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)


  clf2.fit(x_upsampled, y_upsampled)
  y_pred = np.column_stack((y_pred, clf2.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))


  clf3.fit(x1[train_index,:], y.iloc[train_index])
  y_pred = np.column_stack((y_pred, clf3.predict_proba(x1[val_index,:])[:,-1].reshape(-1,1)))


  clf4.fit(x3[train_index,:], y.iloc[train_index])
  y_pred = np.column_stack((y_pred, clf4.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))

  
  clf5.fit(x3[train_index,:], y.iloc[train_index])
  y_pred = np.column_stack((y_pred, clf5.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))


  clf6.fit(x3[train_index,:], y.iloc[train_index], w_array)
  y_pred = np.column_stack((y_pred, clf6.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))


  clf7.fit(x3[train_index,:], y.iloc[train_index], 
                      eval_set=[(x3[val_index,:], y.iloc[val_index])], early_stopping_rounds=10, verbose=False, sample_weight=w_array)
  y_pred = np.column_stack((y_pred, clf7.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))  


  clf8.fit(x3[train_index,:], y.iloc[train_index], 
                      eval_set=[(x3[val_index,:], y.iloc[val_index])], early_stopping_rounds=10, verbose=False, sample_weight=w_array)
  y_pred = np.column_stack((y_pred, clf8.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))  

  


  clf9.fit(x3[train_index,:], y.iloc[train_index], w_array)
  y_pred = np.column_stack((y_pred, clf9.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))



  clf10.fit(x3[train_index,:], y.iloc[train_index], w_array)
  y_pred = np.column_stack((y_pred, clf10.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))



  # clf11.fit(x4[train_index,:], y.iloc[train_index], w_array)
  # y_pred = np.column_stack((y_pred, clf11.predict_proba(x4[val_index,:])[:,-1].reshape(-1,1)))


  clf12.fit(x4[train_index,:], y.iloc[train_index], w_array)
  y_pred = np.column_stack((y_pred, clf12.predict_proba(x4[val_index,:])[:,-1].reshape(-1,1)))


  clf13.fit(x4[train_index,:], y.iloc[train_index], w_array)
  y_pred = np.column_stack((y_pred, clf13.predict_proba(x4[val_index,:])[:,-1].reshape(-1,1)))


  # clf14.fit(x5[train_index,:], y.iloc[train_index], w_array)
  # y_pred = np.column_stack((y_pred, clf14.predict_proba(x5[val_index,:])[:,-1].reshape(-1,1)))


  # clf15.fit(x5[train_index,:], y.iloc[train_index], w_array)
  # y_pred = np.column_stack((y_pred, clf15.predict_proba(x5[val_index,:])[:,-1].reshape(-1,1)))


  # clf16.fit(x5[train_index,:], y.iloc[train_index], w_array)
  # y_pred = np.column_stack((y_pred, clf16.predict_proba(x5[val_index,:])[:,-1].reshape(-1,1)))


  # clf17.fit(x1[train_index,:], y.iloc[train_index], w_array)
  # y_pred = np.column_stack((y_pred, clf17.predict_proba(x1[val_index,:])[:,-1].reshape(-1,1)))


  eval_pool = Pool(x3[val_index,:], y.iloc[val_index])
  clf18.fit(x3[train_index,:], y.iloc[train_index], eval_set=eval_pool, early_stopping_rounds=10, verbose=False)
  y_pred = np.column_stack((y_pred, clf18.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))
  

  es = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
  label_train = y.iloc[train_index]
  label_test = y.iloc[val_index]
  label_train = label_train.values
  label_test = label_test.values

  clf19.fit(x2[train_index,:], label_train, batch_size=1000, epochs=1000, 
              validation_data=(x2[val_index,:], label_test), callbacks=[es], verbose=0, class_weight=class_weights)
  y_pred = np.column_stack((y_pred, clf19.predict(x2[val_index,:])[:,-1].reshape(-1,1)))


  clf20.fit(x2[train_index,:], label_train, batch_size=1000, epochs=1000, 
              validation_data=(x2[val_index,:], label_test), callbacks=[es], verbose=0, class_weight=class_weights)
  y_pred = np.column_stack((y_pred, clf20.predict(x2[val_index,:])[:,-1].reshape(-1,1)))


  clf21.fit(x2[train_index,:], label_train, batch_size=1000, epochs=1000, 
              validation_data=(x2[val_index,:], label_test), callbacks=[es], verbose=0, class_weight=class_weights)
  y_pred = np.column_stack((y_pred, clf21.predict(x2[val_index,:])[:,-1].reshape(-1,1)))


  if first_operation:
    out_of_fold_predictions = y_pred
    first_operation = False
  else:
    out_of_fold_predictions = np.row_stack((out_of_fold_predictions, y_pred))




print('second stacking')
first_operation = True
x1 = np.column_stack((x1, out_of_fold_predictions, np.mean(out_of_fold_predictions, axis=1).reshape(-1,1)))
x2 = np.column_stack((x2, out_of_fold_predictions, np.mean(out_of_fold_predictions, axis=1).reshape(-1,1)))


for i, (train_index, val_index) in enumerate(kfold.split(x, y)):
  w_array = compute_sample_weight('balanced', y.iloc[train_index])
  print(f'\n\t{i+1} cv')
  sclf1.fit(x3[train_index,:], y.iloc[train_index], 
                        eval_set=[(x3[val_index,:], y.iloc[val_index])], early_stopping_rounds=10, verbose=False, sample_weight=w_array)
  y_pred =  sclf1.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)


  eval_pool = Pool(x3[val_index,:], y.iloc[val_index])
  sclf2.fit(x3[train_index,:], y.iloc[train_index], eval_set=eval_pool, early_stopping_rounds=10, verbose=False)
  y_pred = np.column_stack((y_pred, sclf2.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))
  


  sclf3.fit(x3[train_index,:], y.iloc[train_index], 
                      eval_set=[(x3[val_index,:], y.iloc[val_index])], early_stopping_rounds=10, verbose=False, sample_weight=w_array)
  y_pred = np.column_stack((y_pred, sclf3.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))  
  


  es = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
  label_train = y.iloc[train_index]
  label_test = y.iloc[val_index]
  label_train = label_train.values
  label_test = label_test.values

  sclf4.fit(x2[train_index,:], label_train, batch_size=1000, epochs=1000, 
              validation_data=(x2[val_index,:], label_test), callbacks=[es], verbose=0, class_weight=class_weights)
  y_pred = np.column_stack((y_pred, sclf4.predict(x2[val_index,:])[:,-1].reshape(-1,1)))
  

  sclf5.fit(x2[train_index,:], label_train, batch_size=1000, epochs=1000, 
              validation_data=(x2[val_index,:], label_test), callbacks=[es], verbose=0, class_weight=class_weights)
  y_pred = np.column_stack((y_pred, sclf5.predict(x2[val_index,:])[:,-1].reshape(-1,1)))
  
  


  sclf6.fit(x2[train_index,:], label_train, batch_size=1000, epochs=1000, 
              validation_data=(x2[val_index,:], label_test), callbacks=[es], verbose=0, class_weight=class_weights)
  y_pred = np.column_stack((y_pred, sclf6.predict(x2[val_index,:])[:,-1].reshape(-1,1)))
  
  
  sclf7.fit(x1[train_index,:], y.iloc[train_index])
  y_pred = np.column_stack((y_pred, sclf7.predict_proba(x1[val_index,:])[:,-1].reshape(-1,1))) 
  

  if first_operation:
    out_of_fold_predictions = y_pred
    first_operation = False
  else:
    out_of_fold_predictions = np.row_stack((out_of_fold_predictions, y_pred))


  
first_operation = True
print('third stack')
x1 = np.column_stack((x1, out_of_fold_predictions, np.mean(out_of_fold_predictions, axis=1).reshape(-1, 1)))
x3 = np.column_stack((x3, out_of_fold_predictions, np.mean(out_of_fold_predictions, axis=1).reshape(-1, 1)))

for i, (train_index, val_index) in enumerate(kfold.split(x, y)):
  w_array = compute_sample_weight('balanced', y.iloc[train_index])
  print(f'\n\t{i+1} cv')
  ssclf1.fit(x3[train_index,:], y.iloc[train_index], 
                        eval_set=[(x3[val_index,:], y.iloc[val_index])], early_stopping_rounds=10, verbose=False, sample_weight=w_array)
  y_pred =  ssclf1.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)


  eval_pool = Pool(x3[val_index,:], y.iloc[val_index])
  ssclf2.fit(x3[train_index,:], y.iloc[train_index], eval_set=eval_pool, early_stopping_rounds=10, verbose=False)
  y_pred = np.column_stack((y_pred, ssclf2.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))
  


  ssclf3.fit(x3[train_index,:], y.iloc[train_index], 
                      eval_set=[(x3[val_index,:], y.iloc[val_index])], early_stopping_rounds=10, verbose=False, sample_weight=w_array)
  y_pred = np.column_stack((y_pred, ssclf3.predict_proba(x3[val_index,:])[:,-1].reshape(-1,1)))  


  ssclf4.fit(x1[train_index,:], y.iloc[train_index])
  y_pred = np.column_stack((y_pred, ssclf4.predict_proba(x1[val_index,:])[:,-1].reshape(-1,1))) 

  if first_operation:
    out_of_fold_predictions = y_pred
    first_operation = False
  else:
    out_of_fold_predictions = np.row_stack((out_of_fold_predictions, y_pred))




# fourth level stack
print('fourth stack')
x3 = np.column_stack((x3, out_of_fold_predictions, np.mean(out_of_fold_predictions, axis=1).reshape(-1, 1)))
print(x3.shape)
eval_pool = Pool(x3[val_index,:], y.iloc[val_index])
sclf.fit(x3[train_index,:], y.iloc[train_index], eval_set=eval_pool, early_stopping_rounds=10, verbose=False)
print('finished')

In [0]:
import pickle

pickle.dump(clf1, open('/content/drive/My Drive/fraud detection/models/first_stack/clf1_knn_distance', 'wb'))
pickle.dump(clf2, open('/content/drive/My Drive/fraud detection/models/first_stack/clf2_knn_uniform', 'wb'))
pickle.dump(clf3, open('/content/drive/My Drive/fraud detection/models/first_stack/clf3_log_reg', 'wb'))
pickle.dump(clf4, open('/content/drive/My Drive/fraud detection/models/first_stack/clf4_extra_trees', 'wb'))
pickle.dump(clf5, open('/content/drive/My Drive/fraud detection/models/first_stack/clf5_random_forest', 'wb'))
pickle.dump(clf6, open('/content/drive/My Drive/fraud detection/models/first_stack/clf6_bernoulli', 'wb'))
pickle.dump(clf7, open('/content/drive/My Drive/fraud detection/models/first_stack/clf7_lgbm', 'wb'))
pickle.dump(clf8, open('/content/drive/My Drive/fraud detection/models/first_stack/clf8_xgb', 'wb'))
pickle.dump(clf9, open('/content/drive/My Drive/fraud detection/models/first_stack/clf9_gradient_boosting', 'wb'))
pickle.dump(clf10, open('/content/drive/My Drive/fraud detection/models/first_stack/clf10_adaboost', 'wb'))
pickle.dump(clf12, open('/content/drive/My Drive/fraud detection/models/first_stack/clf12_rbf_svm', 'wb'))
pickle.dump(clf13, open('/content/drive/My Drive/fraud detection/models/first_stack/clf13_poly_svm', 'wb'))
clf18.save_model('/content/drive/My Drive/fraud detection/models/first_stack/clf18_cat')
pickle.dump(clf19, open('/content/drive/My Drive/fraud detection/models/first_stack/clf19_nn1', 'wb'))
pickle.dump(clf20, open('/content/drive/My Drive/fraud detection/models/first_stack/clf20_nn2', 'wb'))
pickle.dump(clf21, open('/content/drive/My Drive/fraud detection/models/first_stack/clf21_nn3', 'wb'))

In [0]:
pickle.dump(sclf1, open('/content/drive/My Drive/fraud detection/models/second_stack/sclf1_lgbm', 'wb'))
pickle.dump(sclf2, open('/content/drive/My Drive/fraud detection/models/second_stack/sclf2_cat', 'wb'))
pickle.dump(sclf3, open('/content/drive/My Drive/fraud detection/models/second_stack/sclf3_xgb', 'wb'))
pickle.dump(sclf4, open('/content/drive/My Drive/fraud detection/models/second_stack/sclf4_nn1', 'wb'))
pickle.dump(sclf5, open('/content/drive/My Drive/fraud detection/models/second_stack/sclf5_nn2', 'wb'))
pickle.dump(sclf6, open('/content/drive/My Drive/fraud detection/models/second_stack/sclf6_nn3', 'wb'))
pickle.dump(sclf7, open('/content/drive/My Drive/fraud detection/models/second_stack/sclf7_log_reg', 'wb'))

pickle.dump(ssclf1, open('/content/drive/My Drive/fraud detection/models/third_stack/ssclf1_lgbm', 'wb'))
pickle.dump(ssclf2, open('/content/drive/My Drive/fraud detection/models/third_stack/ssclf2_cat', 'wb'))
pickle.dump(ssclf3, open('/content/drive/My Drive/fraud detection/models/third_stack/ssclf3_xgb', 'wb'))
pickle.dump(ssclf4, open('/content/drive/My Drive/fraud detection/models/third_stack/ssclf4_log_reg', 'wb'))

pickle.dump(sclf, open('/content/drive/My Drive/fraud detection/models/fourth_stack/sclf_cat', 'wb'))

In [0]:
def stack_predict(x, return_prob=False):
  x1 = preprocesser1.transform(x)
  x2 = preprocesser2.transform(x).toarray()
  x3 = preprocesser3.transform(x)
  x4 = preprocesser4.transform(x).toarray()
  x5 = preprocesser5.transform(x).toarray()

  print('First stack')
  y_pred = clf1.predict_proba(x3)[:,-1].reshape(-1,1)
  for clf in [clf2, clf3, clf4, clf5, clf6, clf7, clf8, clf9, clf10, clf12, clf13, clf18, clf19, clf20, clf21]:
    if clf in [clf3]:
      y_pred = np.column_stack((y_pred, clf.predict_proba(x1)[:,-1].reshape(-1,1)))
    elif clf in [clf19, clf20, clf21]:
      y_pred = np.column_stack((y_pred, clf.predict(x2)[:,-1].reshape(-1,1)))
    elif clf in [ clf12, clf13]:
      y_pred = np.column_stack((y_pred, clf.predict_proba(x4)[:,-1].reshape(-1,1)))
    else:
      y_pred = np.column_stack((y_pred, clf.predict_proba(x3)[:,-1].reshape(-1,1)))


  print('Second stack')
  x1 = np.column_stack((x1, y_pred, np.mean(y_pred, axis=1).reshape(-1,1)))
  x2 = np.column_stack((x2, y_pred, np.mean(y_pred, axis=1).reshape(-1,1)))
  x3 = np.column_stack((x3, y_pred, np.mean(y_pred, axis=1).reshape(-1,1)))

  y_pred = sclf1.predict_proba(x3)[:,-1].reshape(-1,1)
  for clf in [sclf2, sclf3, sclf4, sclf5, sclf6, sclf7]:
    if clf in [sclf7]:
      y_pred = np.column_stack((y_pred, clf.predict_proba(x1)[:,-1].reshape(-1,1)))
    elif clf in [sclf4, sclf5, sclf6]:
      y_pred = np.column_stack((y_pred, clf.predict(x2)[:,-1].reshape(-1,1)))
    else:
      y_pred = np.column_stack((y_pred, clf.predict_proba(x3)[:,-1].reshape(-1,1)))


  print('Third stack')
  x1 = np.column_stack((x1, y_pred, np.mean(y_pred, axis=1).reshape(-1,1)))
  x3 = np.column_stack((x3, y_pred, np.mean(y_pred, axis=1).reshape(-1,1)))
  y_pred = ssclf1.predict_proba(x3)[:,-1].reshape(-1,1)
  for clf in [ssclf2, ssclf3, ssclf4]:
    if clf in [ssclf4]:
      y_pred = np.column_stack((y_pred, clf.predict_proba(x1)[:,-1].reshape(-1,1)))
    else:
      y_pred = np.column_stack((y_pred, clf.predict_proba(x3)[:,-1].reshape(-1,1)))

  print('Fourth stack')
  x3 = np.column_stack((x3, y_pred, np.mean(y_pred, axis=1).reshape(-1,1)))
  if return_proba:
    test_pred = sclf.predict_proba(x3)
  else:
    test_pred = sclf.predict(x3)

In [0]:
test = pd.read_csv("/content/drive/My Drive/fraud detection/data/processed_test.csv")
test.head()

,account destination verification,account source verification,account type,age,credit card type,current bank amount,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,account type_last bank amount_min,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)
0,0,1,current,78,verve,88677,bcus2993,1059697,unknown,29964,accounting,3033,1,1,2016,2,6,26,28,12,51,43,1,30,10,-971020,-1029733,-58713,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,8,...,30,49,277340,-0.016715,-1.188783,14.394111,5056,-703042.104628,97142,-1489412,1586554,-3554580881,0.017912,-1.183684,434988.770042,5056,-502333.386076,481031,-1467547,1948578,-2539797600,0.004821,-1.006667,455073.325606,5056,200708.718552,495223,-96346,591569,1014783281,0.000620,-1.104122,147897.801722,5056,29.237389,349.389054,9.879327,-320.151665,-339.509726,-58713
1,0,1,current,38,master,85737,bcus2994,434908,single,354835,farmer,3025,0,1,2015,4,12,51,16,18,4,3,2,31,2,-349171,-80073,269098,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,8,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,28.342810,143.771240,117.300826,-115.428430,-26.470413,269098
2,0,1,current,47,master,14611,bcus2995,1287631,unknown,395445,bank

In [0]:
test = pd.read_csv('/content/drive/My Drive/fraud detection/data/test.csv')

In [0]:
test.head()

,id,transaction time,current bank amount,last bank amount,time taken (seconds),most recent bank amount,account type,marital_status,age,occupation,credit card type,account source verification,transaction source method,account destination verification
0,bcus2993,2016-06-28T12:51:43,88677,1059697,3033,29964,current,unknown,78,accounting,verve,True,True,False
1,bcus2994,2015-12-16T18:04:03,85737,434908,3025,354835,current,single,38,farmer,master,True,False,False
2,bcus2995,2016-10-15T04:06:55,14611,1287631,2414,395445,current,unknown,47,banking,master,True,False,False
3,bcus2996,2014-03-15T04:40:45,53130,61604,1228,204872,current,single,74,clergy,master,False,False,True
4,bcus2997,2015-05-25T05:09:33,20643,1143031,1031,207262,current,married,72,lawyer,master,True,False,True


In [0]:
test_labels = np.round(sclf.predict(x3)).flatten()

In [0]:
submission = pd.DataFrame({'id':test['id'],'fradulent':test_labels})
submission.head(10)

,id,fradulent
0,bcus2993,0
1,bcus2994,1
2,bcus2995,0
3,bcus2996,1
4,bcus2997,0
5,bcus2998,0
6,bcus2999,0
7,bcus3000,0
8,bcus3001,1
9,bcus3002,1


In [0]:
submission.to_csv('submission.csv',index=False)

In [0]:
x = pickle.load(open('/content/drive/My Drive/fraud detection/models/out_of_fold_predictions', 'rb'))
lr = LogisticRegression(class_weight='balanced')
lr.fit(x,y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# x = test
# x1 = preprocesser1.transform(x)
# x2 = preprocesser2.transform(x).toarray()
# x3 = preprocesser3.transform(x)
# x4 = preprocesser4.transform(x).toarray()
# x5 = preprocesser5.transform(x).toarray()
# y_pred = clf1.predict_proba(x3)[:,-1].reshape(-1,1)
# for clf in [clf2, clf3, clf4, clf5, clf6, clf7, clf8, clf9, clf10, clf12, clf13, clf18, clf19, clf20, clf21]:
#   if clf in [clf3]:
#     y_pred = np.column_stack((y_pred, clf.predict_proba(x1)[:,-1].reshape(-1,1)))
#   elif clf in [clf19, clf20, clf21]:
#     y_pred = np.column_stack((y_pred, clf.predict(x2)[:,-1].reshape(-1,1)))
#   elif clf in [ clf12, clf13]:
#     y_pred = np.column_stack((y_pred, clf.predict_proba(x4)[:,-1].reshape(-1,1)))
  # else:
  #   y_pred = np.column_stack((y_pred, clf.predict_proba(x3)[:,-1].reshape(-1,1)))

test_pred = lr.predict(y_pred)

In [0]:
test_labels = np.round(test_pred).flatten()

In [0]:
submission = pd.DataFrame({'id':test['id'],'fradulent':test_labels})
submission.head(10)

,id,fradulent
0,bcus2993,0
1,bcus2994,1
2,bcus2995,1
3,bcus2996,0
4,bcus2997,1
5,bcus2998,0
6,bcus2999,0
7,bcus3000,0
8,bcus3001,1
9,bcus3002,0


In [0]:
submission.to_csv('submission3.csv',index=False)